In [4]:
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession

In [3]:
from pyspark.ml.recommendation import ALS

In [ ]:
../../data/ml-latest-small/movies.csv"

In [22]:
movies = pd.read_csv('../../data/ml-latest-small/movies.csv')
ratings = pd.read_csv('../../data/ml-latest-small/ratings.csv')
links = pd.read_csv('../../data/ml-latest-small/links.csv')
tags = pd.read_csv('../../data/ml-latest-small/tags.csv')

In [5]:
pwd

'/home/jovyan/work/Desktop/dsi_lax3/assignments/recommender-big-four/src/pyspark'

In [23]:
movies.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [24]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224


In [25]:
links.head(3)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0


In [26]:
tags.head(3)

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992


In [29]:
len(tags.tag.value_counts())

1589

In [31]:
spark = SparkSession.builder.getOrCreate()

In [32]:
spark_ratings = spark.createDataFrame(ratings)
spark_ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

